## Part 1: Preprocessing

In [205]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
display(attrition_df)
attrition_df['Department'].value_counts()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Research & Development,23,2,Medical,3,41,4,...,3,3,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,Research & Development,6,1,Medical,4,42,2,...,3,1,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,Research & Development,4,3,Life Sciences,2,87,4,...,4,2,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,Sales,2,3,Medical,4,63,2,...,3,4,0,17,3,2,9,6,0,8


Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [206]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [207]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [208]:
# Create a list of at least 10 column names to use as X data
columns = ['Age','DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'NumCompaniesWorked']
           
# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [209]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [210]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import OneHotEncoder
OverTime_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
OverTime_encoder.fit(np.array(X_train['OverTime']).reshape(-1,1))
X_train['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_train['OverTime']).reshape(-1,1))
X_test['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_test['OverTime']).reshape(-1,1))
X_train = X_train.drop('OverTime', axis=1)
X_test = X_test.drop('OverTime', axis=1)

In [211]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [212]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train_encoded = department_encoder.transform(np.array(y_train['Department']).reshape(-1,1))
y_department_test_encoded = department_encoder.transform(np.array(y_test['Department']).reshape(-1,1))
y_department_train_encoded


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [213]:
# Create a OneHotEncoder for the Attrition column
from sklearn.preprocessing import OneHotEncoder
attrition_encoded = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoded.fit(np.array(y_train['Attrition']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train_encoded = attrition_encoded.transform(np.array(y_train['Attrition']).reshape(-1,1))
y_attrition_test_encoded = attrition_encoded.transform(np.array(y_test['Attrition']).reshape(-1,1))
y_attrition_train_encoded

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [214]:
# Find the number of columns in the X training data
input_features = len(X_train_scaled[0])

# Create the input layer
input = layers.Input(shape=(input_features,))
print(input_features)

# Create at least two shared layers
shared_1 = layers.Dense(64, activation='relu')(input)
shared_2 = layers.Dense(128, activation='relu')(shared_1)

10


In [215]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer_department = layers.Dense(32, activation='relu')(shared_2)

# Create the output layer
department_output = layers.Dense(3, activation='sigmoid', name='department_output')(hidden_layer_department)

In [216]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer_attrition = layers.Dense(32, activation='relu')(shared_2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train_encoded.shape[1], activation='sigmoid', name='attrition_output')(hidden_layer_attrition)

In [217]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })

# Summarize the model
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 dense_45 (Dense)               (None, 64)           704         ['input_10[0][0]']               
                                                                                                  
 dense_46 (Dense)               (None, 128)          8320        ['dense_45[0][0]']               
                                                                                                  
 dense_47 (Dense)               (None, 32)           4128        ['dense_46[0][0]']               


                                                                                                  
 dense_48 (Dense)               (None, 32)           4128        ['dense_46[0][0]']               
                                                                                                  
 department_output (Dense)      (None, 3)            99          ['dense_47[0][0]']               
                                                                                                  
 attrition_output (Dense)       (None, 2)            66          ['dense_48[0][0]']               
                                                                                                  
Total params: 17,445
Trainable params: 17,445
Non-trainable params: 0
__________________________________________________________________________________________________


In [218]:
import tensorflow as tf
tf.__version__

'2.10.1'

In [219]:
# Train the model
fit_model = model.fit(X_train_scaled,{'department_output': y_department_train_encoded, 'attrition_output': y_attrition_train_encoded},epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 2ms/step - loss: 1.3425 - department_output_loss: 0.8135 - attrition_output_loss: 0.5290 - department_output_accuracy: 0.6379 - attrition_output_accuracy: 0.8049
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: 1.1865 - department_output_loss: 0.7567 - attrition_output_loss: 0.4298 - department_output_accuracy: 0.6670 - attrition_output_accuracy: 0.8285
Epoch 3/100
35/35 [==============================] - 0s 2ms/step - loss: 1.1415 - department_output_loss: 0.7435 - attrition_output_loss: 0.3979 - department_output_accuracy: 0.6679 - attrition_output_accuracy: 0.8448
Epoch 4/100
35/35 [==============================] - 0s 2ms/step - loss: 1.1174 - department_output_loss: 0.7318 - attrition_output_loss: 0.3856 - department_output_accuracy: 0.6688 - attrition_output_accuracy: 0.8494
Epoch 5/100
35/35 [==============================] - 0s 2ms/step - loss: 1.0949 - department_output_loss: 0.7231 - attrition_out

In [223]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {'department_output': y_department_test_encoded, 'attrition_output': y_attrition_test_encoded})

# Unpack the results
model_loss = results[0]
department_loss = results[1]
attrition_loss = results[2]
department_accuracy = results[3]
attrition_accuracy = results[4]

print(f"Model Loss: {model_loss}")
print(f"Department Loss: {department_loss}")
print(f"Attrition Loss: {attrition_loss}")
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

12/12 [==============================] - 0s 1ms/step - loss: 4.1812 - department_output_loss: 3.3184 - attrition_output_loss: 0.8628 - department_output_accuracy: 0.5000 - attrition_output_accuracy: 0.8424
Model Loss: 4.181197643280029
Department Loss: 3.3183939456939697
Attrition Loss: 0.8628035187721252
Department Accuracy: 0.5
Attrition Accuracy: 0.842391312122345


In [225]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")


Department Accuracy: 0.5
Attrition Accuracy: 0.842391312122345


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 